# Prepare COVID-19 data from Robert Koch-Institute

## 1. Prepare spatial data

RKI COVID data are reported per district (**LK = "Landkreis"**), but do not contain coordinates.
Therefore, we read an LK polygon shapefile, and compute the center coordinate for each polygon.
These coordinates are then appended to the COVID data using the Landkreis ID.

Data source: [Robert Koch-Institut (RKI), dl-de/by-2-0](https://npgeo-corona-npgeo-de.hub.arcgis.com/datasets/917fc37a709542548cc3be077a786c17_0)

In [1]:
require(rgdal) # for reading shapefile
require(rgeos) # for computing centroids

# read Landkreis Shapefile
lk <- readOGR("data/Landkreise.shp")
str(lk@data)

# compute centroid per polygon
cen <- gCentroid(lk, byid=TRUE)
#str(cen)

# make table with coordinates and Landkreis ID
lk_ <- cbind(as.data.frame(cen@coords), RS=as.integer(gsub("^0", "", lk$RS)), POP=lk$EWZ)
str(lk_)

Loading required package: rgdal

Loading required package: sp

rgdal: version: 1.4-7, (SVN revision 845)
 Geospatial Data Abstraction Library extensions to R successfully loaded
 Loaded GDAL runtime: GDAL 2.2.2, released 2017/09/15
 Path to GDAL shared files: /usr/share/gdal/2.2
 GDAL binary built with GEOS: TRUE 
 Loaded PROJ.4 runtime: Rel. 4.9.2, 08 September 2015, [PJ_VERSION: 492]
 Path to PROJ.4 shared files: (autodetected)
 Linking to sp version: 1.3-2 

Loading required package: rgeos

rgeos version: 0.5-3, (SVN revision 634)
 GEOS runtime version: 3.5.1-CAPI-1.9.1 
 Linking to sp version: 1.3-2 
 Polygon checking: TRUE 




OGR data source with driver: ESRI Shapefile 
Source: "/home/frantzda/cor/covid19/data/Landkreise.shp", layer: "Landkreise"
with 412 features
It has 39 fields
'data.frame':	412 obs. of  39 variables:
 $ FID       : int  1 2 3 4 5 6 7 8 9 10 ...
 $ ADE       : int  4 4 4 4 4 4 4 4 4 4 ...
 $ GF        : int  4 4 4 4 4 4 4 4 4 4 ...
 $ BSG       : int  1 1 1 1 1 1 1 1 1 1 ...
 $ RS        : Factor w/ 412 levels "01001","01002",..: 1 2 3 4 5 6 7 8 9 10 ...
 $ AGS       : Factor w/ 400 levels "01001","01002",..: 1 2 3 4 5 6 7 8 9 10 ...
 $ SDV_RS    : Factor w/ 368 levels "010010000000",..: 1 2 3 4 5 6 7 8 9 10 ...
 $ GEN       : Factor w/ 390 levels "Ahrweiler","Aichach-Friedberg",..: 104 170 193 229 77 151 237 261 268 270 ...
 $ BEZ       : Factor w/ 5 levels "Bezirk","Kreis",..: 3 3 3 3 2 2 2 2 2 2 ...
 $ IBZ       : int  40 40 40 40 42 42 42 42 42 42 ...
 $ BEM       : Factor w/ 2 levels "--","Sonderverband": 1 1 1 1 1 1 1 1 1 1 ...
 $ NBD       : Factor w/ 2 levels "ja","nein": 1 1 1 1

## 2. Prepare COVID-19 data

### 2.1 Download and clean data

This code snippet downloads and prepares the **latest COVID-19 data** from RKI.
RKI updates the COVID-19 cases on a daily basis.

The cases are reported for districts ("Landkreise") - with the exception of Berlin, where cases are reported for the 12 "Bezirke".
To match the spatial data, Berlin is dissolved to the district level.

Data source: [Robert Koch-Institut (RKI), dl-de/by-2-0](https://npgeo-corona-npgeo-de.hub.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0)

In [2]:
# download latest data
download.file("https://opendata.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0.csv", "data/RKI_COVID19.csv")

# read COVID-19 statistics
rki <- read.csv("data/RKI_COVID19.csv")
#str(rki)

# add new column for Landkreis ID
rki <- cbind(rki, RS=as.integer(rki$IdLandkreis))
#str(rki)

# dissolve Berlin districts
#rki$RS[which(rki$RS > 11000 & rki$RS < 12000)] <- 11000L
#rki$Landkreis <- gsub("(SK Berlin).*", "\\1", rki$Landkreis)

# fix -1 cases
# not sure why this happens, but if these values are set to 1, the cases are identical to the ones reported in the news
rki$AnzahlFall[which(rki$AnzahlFall < 1)] <- 1

# add new column with a proper date
rki <- cbind(rki, date=as.POSIXct(rki$Meldedatum))
str(rki)


'data.frame':	145408 obs. of  20 variables:
 $ FID                 : int  9819932 9819933 9819934 9819935 9819936 9819937 9819938 9819939 9819940 9819941 ...
 $ IdBundesland        : int  1 1 1 1 1 1 1 1 1 1 ...
 $ Bundesland          : Factor w/ 16 levels "Baden-Württemberg",..: 15 15 15 15 15 15 15 15 15 15 ...
 $ Landkreis           : Factor w/ 412 levels "LK Ahrweiler",..: 336 336 336 336 336 336 336 336 336 336 ...
 $ Altersgruppe        : Factor w/ 7 levels "A00-A04","A05-A14",..: 3 3 3 3 3 3 3 3 3 3 ...
 $ Geschlecht          : Factor w/ 3 levels "M","unbekannt",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ AnzahlFall          : num  1 1 1 1 1 1 1 1 1 1 ...
 $ AnzahlTodesfall     : int  0 0 0 0 0 0 0 0 0 0 ...
 $ Meldedatum          : Factor w/ 109 levels "2020/01/28 00:00:00",..: 31 36 36 38 44 52 54 55 57 107 ...
 $ IdLandkreis         : int  1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 ...
 $ Datenstand          : Factor w/ 1 level "01.06.2020, 00:00 Uhr": 1 1 1 1 1 1 1 1 1 1 ...
 $ Ne

### 2.2 Convert to dataframe

The RKI data are very detailed.
For keeping things simple, we only keep necessary variables:
- coordinates (Lat/Lon)
- time
- Landkreis name and ID
- population

If all properties (e.g. district, date, age, gender etc.) are the same, RKI seems to group cases.
However, we need the number of cases per district and day (but regardless of age or gender).
1. We firstly explode these groups into individual cases.
   The dataframe ``df`` has one row for each COVID-19 case in Germany.
   Many rows in ``df`` are redundant.
2. We count unique rows.
   The new dataframe ``df_day`` has one row per district and day (with case count as extra column).
3. We compute incidence (cases per 100,000 cap)

In [3]:
require(plyr) # count cases for each LK and day

# compile dataframe
# sometimes, cases are reported together -> explode
df <- data.frame(X=0, 
                 Y=0, 
                 T=rep(rki$date,       rki$AnzahlFall),
                 LK=rep(rki$Landkreis, rki$AnzahlFall),
                 ID=rep(rki$RS,        rki$AnzahlFall),
                 POP=0)
pos <- sapply(df$ID, function(x)which(x==lk_$RS))
df$X   <- lk_$x[pos]
df$Y   <- lk_$y[pos]
df$POP <- lk_$POP[pos]
str(df)
              
# compile analysis-ready data
df_day <- count(df)
colnames(df_day)[7] <- "N"
str(df_day)

Loading required package: plyr



'data.frame':	181837 obs. of  6 variables:
 $ X  : num  9.44 9.44 9.44 9.44 9.44 ...
 $ Y  : num  54.8 54.8 54.8 54.8 54.8 ...
 $ T  : POSIXct, format: "2020-03-14" "2020-03-19" ...
 $ LK : Factor w/ 412 levels "LK Ahrweiler",..: 336 336 336 336 336 336 336 336 336 336 ...
 $ ID : int  1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 ...
 $ POP: int  89504 89504 89504 89504 89504 89504 89504 89504 89504 89504 ...
'data.frame':	22706 obs. of  7 variables:
 $ X  : num  9.44 9.44 9.44 9.44 9.44 ...
 $ Y  : num  54.8 54.8 54.8 54.8 54.8 ...
 $ T  : POSIXct, format: "2020-03-14" "2020-03-19" ...
 $ LK : Factor w/ 412 levels "LK Ahrweiler",..: 336 336 336 336 336 336 336 336 336 336 ...
 $ ID : int  1001 1001 1001 1001 1001 1001 1001 1001 1001 1001 ...
 $ POP: int  89504 89504 89504 89504 89504 89504 89504 89504 89504 89504 ...
 $ N  : int  4 4 1 1 1 1 1 1 1 2 ...


## 3. Export analysis-ready data

In [4]:
# write table with n cases per day and LK
write.csv(df_day, "data/covid19-deu.csv", row.names=FALSE)